# CS 584-A: Natural Language Processing

## Student information
- Full name: Thanapoom Phatthanaphan
- CWID: 20011296

## Homework 4
### Goals
The goal of HW4 is for you to get hands-on experience of utilizing pre-trained and large language model for the machine reading comprehension task. The questions are open questions and there is no fixed solution. Different data selection and processing, parameter initialization, data split strategies will lead to different predictions and results. Therefore, during the grading, the specific values in the results are not required. It is important that you focus on implementing and setting up the pipelines of applying these models to solve the tasks.

## Task: Machine Reading Comprehension

### 1. Data preparation

**1.1 Data download and preprocessing**

In [1]:
# Import the necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install transformers datasets
from datasets import load_dataset

In [2]:
# Load the dataset
dataset = load_dataset('squad_v2')
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [3]:
# Select 15 question-answer pairs
selected_data = dataset['train'][:15]
selected_data

{'id': ['56be85543aeaaa14008c9063',
  '56be85543aeaaa14008c9065',
  '56be85543aeaaa14008c9066',
  '56bf6b0f3aeaaa14008c9601',
  '56bf6b0f3aeaaa14008c9602',
  '56bf6b0f3aeaaa14008c9603',
  '56bf6b0f3aeaaa14008c9604',
  '56bf6b0f3aeaaa14008c9605',
  '56d43c5f2ccc5a1400d830a9',
  '56d43c5f2ccc5a1400d830aa',
  '56d43c5f2ccc5a1400d830ab',
  '56d43c5f2ccc5a1400d830ac',
  '56d43c5f2ccc5a1400d830ad',
  '56d43ce42ccc5a1400d830b4',
  '56d43ce42ccc5a1400d830b5'],
 'title': ['Beyoncé',
  'Beyoncé',
  'Beyoncé',
  'Beyoncé',
  'Beyoncé',
  'Beyoncé',
  'Beyoncé',
  'Beyoncé',
  'Beyoncé',
  'Beyoncé',
  'Beyoncé',
  'Beyoncé',
  'Beyoncé',
  'Beyoncé',
  'Beyoncé'],
 'context': ['Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Dest

**1.2 Data statistics**
<br>a) Statistics about the max/average/min length of the contexts
<br>b) Statistics about the max/average/min length of the questions
<br>c) Statistics about the max/average/min length of the answer

In [4]:
# Statistics about the context length
context_lengths = [len(item) for item in selected_data['context']]
max_context_length = max(context_lengths)
min_context_length = min(context_lengths)
avg_context_length = round((sum(context_lengths) / len(context_lengths)), 0)

# Statistics about the question length
question_lengths = [len(item) for item in selected_data["question"]]
max_question_length = max(question_lengths)
min_question_length = min(question_lengths)
avg_question_length = round((sum(question_lengths) / len(question_lengths)), 0)

# Statistics about the answer length
answer_lengths = [len(item['text'][0]) for item in selected_data['answers']]
max_answer_length = max(answer_lengths)
min_answer_length = min(answer_lengths)
avg_answer_length = round((sum(answer_lengths) / len(answer_lengths)), 0)

# Create the elements of the statistics table
table_elements = {
    'Type' : ['Context', 'Question', 'Answer'],
    'MaxLength' : [max_context_length, max_question_length, max_answer_length],
    'MinLength' : [min_context_length, min_question_length, min_answer_length],
    'AvgLength' : [avg_context_length, avg_question_length, avg_answer_length]
}

# Print the statistics table
statistics_table = pd.DataFrame(table_elements)
statistics_table

,Type,MaxLength,MinLength,AvgLength
0,Context,694,694,694.0
1,Question,64,30,47.0
2,Answer,19,4,13.0


From the table, we can see that the maximum, minimum, and average length of the context are the same because these 15 question-answer pairs are the questions and answers from the same context.

### 2. Question answering

**2.1 Use the pre-trained RoBERTa model to solve the question answering task on the data**

In [5]:
from transformers import AutoTokenizer, RobertaForQuestionAnswering
import torch

# Load pre-trained RoBERTa tokenizer and model for question answering
tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2", truncation=True, max_length=512, padding="max_length")
model = RobertaForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")

# Loop through each data sample and perform question answering
for i in range(15):
    context = selected_data['context'][i]
    question = selected_data['question'][i]

    # Tokenize input
    inputs = tokenizer(question, context, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)

    # Get the answer start and end indices
    answer_start_index = torch.argmax(outputs.start_logits)
    answer_end_index = torch.argmax(outputs.end_logits)

    # Decode answer tokens
    predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
    decoded_answer = tokenizer.decode(predict_answer_tokens, skip_special_tokens=True)

    # Print the required output
    print(f"Question: {question}")
    print(f"Answer Start Index: {answer_start_index}, Answer End Index: {answer_end_index}")
    print(f"Predicted Answer Tokens: {predict_answer_tokens}")
    print(f"Decoded Answer: {decoded_answer}")
    print()

Question: When did Beyonce start becoming popular?
Answer Start Index: 87, Answer End Index: 89
Predicted Answer Tokens: tensor([ 628, 4525,   29])
Decoded Answer:  late 1990s

Question: What areas did Beyonce compete in when she was growing up?
Answer Start Index: 78, Answer End Index: 80
Predicted Answer Tokens: tensor([6970,    8, 7950])
Decoded Answer:  singing and dancing

Question: When did Beyonce leave Destiny's Child and become a solo singer?
Answer Start Index: 155, Answer End Index: 155
Predicted Answer Tokens: tensor([35153])
Decoded Answer: 2003

Question: In what city and state did Beyonce  grow up? 
Answer Start Index: 72, Answer End Index: 72
Predicted Answer Tokens: tensor([1184])
Decoded Answer:  Texas

Question: In which decade did Beyonce become famous?
Answer Start Index: 89, Answer End Index: 90
Predicted Answer Tokens: tensor([4525,   29])
Decoded Answer:  1990s

Question: In what R&B group was she the lead singer?
Answer Start Index: 104, Answer End Index: 106
P

In [6]:
# Try to change some settings in tokenizer
# Load pre-trained RoBERTa tokenizer and model for question answering
tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2", truncation=False, max_length=128, padding=False) # Change truncation, padding to False, and max_length = 128
model = RobertaForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")

# Loop through each data sample and perform question answering
for i in range(15):
    context = selected_data['context'][i]
    question = selected_data['question'][i]

    # Tokenize input
    inputs = tokenizer(question, context, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)

    # Get the answer start and end indices
    answer_start_index = torch.argmax(outputs.start_logits)
    answer_end_index = torch.argmax(outputs.end_logits)

    # Decode answer tokens
    predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
    decoded_answer = tokenizer.decode(predict_answer_tokens, skip_special_tokens=True)

    # Print the required output
    print(f"Question: {question}")
    print(f"Answer Start Index: {answer_start_index}, Answer End Index: {answer_end_index}")
    print(f"Predicted Answer Tokens: {predict_answer_tokens}")
    print(f"Decoded Answer: {decoded_answer}")
    print()

Question: When did Beyonce start becoming popular?
Answer Start Index: 87, Answer End Index: 89
Predicted Answer Tokens: tensor([ 628, 4525,   29])
Decoded Answer:  late 1990s

Question: What areas did Beyonce compete in when she was growing up?
Answer Start Index: 78, Answer End Index: 80
Predicted Answer Tokens: tensor([6970,    8, 7950])
Decoded Answer:  singing and dancing

Question: When did Beyonce leave Destiny's Child and become a solo singer?
Answer Start Index: 155, Answer End Index: 155
Predicted Answer Tokens: tensor([35153])
Decoded Answer: 2003

Question: In what city and state did Beyonce  grow up? 
Answer Start Index: 72, Answer End Index: 72
Predicted Answer Tokens: tensor([1184])
Decoded Answer:  Texas

Question: In which decade did Beyonce become famous?
Answer Start Index: 89, Answer End Index: 90
Predicted Answer Tokens: tensor([4525,   29])
Decoded Answer:  1990s

Question: In what R&B group was she the lead singer?
Answer Start Index: 104, Answer End Index: 106
P

From trying to change some settings to see any difference, I found that the answers are the same. I think the answers remained the same because the context is not complex that is easy for the model to understand and provide the correct answers.

**2.2 Design the prompts to interact with ChatGPT and solve the question answering task on the data**

---
**Prompt Template for ChatGPT about the selected N question-answer pairs**

Context: "context"

Question 1: "question 1"
<br>Question 2: "question 2"
<br>Question N: "question N"

Please provide the short answer to each question about the context, and also an explanation of the reasoning for that answer.

---
**Interaction with ChatGPT about the selected 15 question-answer pairs**

Context: 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

Question 1: When did Beyonce start becoming popular?
<br>Question 2: What areas did Beyonce compete in when she was growing up?
<br>Question 3: When did Beyonce leave Destiny's Child and become a solo singer?
<br>Question 4: In what city and state did Beyonce  grow up?
<br>Question 5: In which decade did Beyonce become famous?
<br>Question 6: In what R&B group was she the lead singer?
<br>Question 7: What album made her a worldwide known artist?
<br>Question 8: Who managed the Destiny's Child group?
<br>Question 9: When did Beyoncé rise to fame?
<br>Question 10: What role did Beyoncé have in Destiny's Child?
<br>Question 11: What was the first album Beyoncé released as a solo artist?
<br>Question 12: When did Beyoncé release Dangerously in Love?
<br>Question 13: How many Grammy awards did Beyoncé win for her first solo album?
<br>Question 14: What was Beyoncé's role in Destiny's Child?
<br>Question 15: What was the name of Beyoncé's first solo album?

Please provide the short answer to each question about the context, and also an explanation of the reasoning for that answer.

<br>**ChatGPT's outputs:**

**1. When did Beyonce start becoming popular?**
- **Answer:** Beyoncé started becoming popular in the late 1990s.
- **Explanation:** The provided context mentions that Beyoncé rose to fame in the late 1990s as the lead singer of the R&B girl-group Destiny's Child.

**2. What areas did Beyonce compete in when she was growing up?**
- **Answer:** Beyoncé competed in various singing and dancing competitions as a child.
- **Explanation:** The context states that she performed in various singing and dancing competitions during her childhood.

**3. When did Beyonce leave Destiny's Child and become a solo singer?**
- **Answer:** Beyoncé left Destiny's Child during their hiatus, and her debut solo album, Dangerously in Love, was released in 2003.
- **Explanation:** The context mentions that their hiatus saw the release of Beyoncé's debut album in 2003, signifying her transition to a solo career.

**4. In what city and state did Beyonce grow up?**
- **Answer:** Beyoncé grew up in Houston, Texas.
- **Explanation:** The context states that she was born and raised in Houston, Texas.

**5. In which decade did Beyonce become famous?**
- **Answer:** Beyoncé became famous in the late 1990s.
- **Explanation:** The context indicates that she rose to fame in the late 1990s.

**6. In what R&B group was she the lead singer?**
- **Answer:** Beyoncé was the lead singer of the R&B girl-group Destiny's Child.
- **Explanation:** The context explicitly mentions that she was the lead singer of Destiny's Child.

**7. What album made her a worldwide known artist?**
- **Answer:** Dangerously in Love (2003) made Beyoncé a worldwide known artist.
- **Explanation:** The context specifies that this album, released in 2003, established her as a solo artist worldwide.

**8. Who managed the Destiny's Child group?**
- **Answer:** Destiny's Child was managed by her father, Mathew Knowles.
- **Explanation:** The context mentions that the group was managed by Beyoncé's father, Mathew Knowles.

**9. When did Beyoncé rise to fame?**
- **Answer:** Beyoncé rose to fame in the late 1990s.
- **Explanation:** The context states that she rose to fame in the late 1990s as the lead singer of Destiny's Child.

**10. What role did Beyoncé have in Destiny's Child?**
- **Answer:** Beyoncé was the lead singer of Destiny's Child.
- **Explanation:** The context explicitly mentions that she was the lead singer of the R&B girl-group Destiny's Child.

**11. What was the first album Beyoncé released as a solo artist?**
- **Answer:** Beyoncé's first solo album was Dangerously in Love (2003).
- **Explanation:** The context specifies that Dangerously in Love (2003) was her debut album as a solo artist.

**12. When did Beyoncé release Dangerously in Love?**
- **Answer:** Beyoncé released Dangerously in Love in 2003.
- **Explanation:** The context provides the release year of her debut solo album.

**13. How many Grammy awards did Beyoncé win for her first solo album?**
- **Answer:** Beyoncé won five Grammy Awards for her first solo album, Dangerously in Love.
- **Explanation:** The context mentions that her debut solo album, Dangerously in Love (2003), earned her five Grammy Awards.

**14. What was Beyoncé's role in Destiny's Child?**
- **Answer:** Beyoncé was the lead singer of Destiny's Child.
- **Explanation:** The context explicitly states that she was the lead singer of the R&B girl-group Destiny's Child.

**15. What was the name of Beyoncé's first solo album?**
- **Answer:** Beyoncé's first solo album was titled Dangerously in Love.
- **Explanation:** The context provides the title of her debut solo album.

You can see the pictures} of interaction with chatgpt in the link:
https://drive.google.com/drive/folders/111jspKOhyvRvClOTLV4Aj8tli--pKaby?usp=sharing

**2.3 Design the prompts to interact with Claude 2 and solve the question answering task on the data**

---
**Prompt Template for Claude 2 about the selected N question-answer pairs (Same as the template used for ChatGPT)**

Context: "context"

Question 1: "question 1"
<br>Question 2: "question 2"
<br>Question N: "question N"

Please provide the short answer to each question about the context, and also an explanation of the reasoning for that answer.

---
**Interaction with Claude 2 about the selected 15 question-answer pairs**

Context: 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

Question 1: When did Beyonce start becoming popular?
<br>Question 2: What areas did Beyonce compete in when she was growing up?
<br>Question 3: When did Beyonce leave Destiny's Child and become a solo singer?
<br>Question 4: In what city and state did Beyonce  grow up?
<br>Question 5: In which decade did Beyonce become famous?
<br>Question 6: In what R&B group was she the lead singer?
<br>Question 7: What album made her a worldwide known artist?
<br>Question 8: Who managed the Destiny's Child group?
<br>Question 9: When did Beyoncé rise to fame?
<br>Question 10: What role did Beyoncé have in Destiny's Child?
<br>Question 11: What was the first album Beyoncé released as a solo artist?
<br>Question 12: When did Beyoncé release Dangerously in Love?
<br>Question 13: How many Grammy awards did Beyoncé win for her first solo album?
<br>Question 14: What was Beyoncé's role in Destiny's Child?
<br>Question 15: What was the name of Beyoncé's first solo album?

Please provide the short answer to each question about the context, and also an explanation of the reasoning for that answer.

<br>**Claude 2's outputs:**
1. When did Beyonce start becoming popular?

She rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child.

2. What areas did Beyonce compete in when she was growing up?

The context states she performed in various singing and dancing competitions as a child.

3. When did Beyonce leave Destiny's Child and become a solo singer?  

The context states that during Destiny's Child's hiatus, Beyoncé released her debut solo album Dangerously in Love in 2003. So she left the group and went solo in 2003.

4. In what city and state did Beyonce grow up?

She was born and raised in Houston, Texas.

5. In which decade did Beyonce become famous?  

She became famous in the late 1990s.

6. In what R&B group was she the lead singer?

Destiny's Child

7. What album made her a worldwide known artist?

Her debut solo album "Dangerously in Love" in 2003 established her as a solo artist worldwide.  

8. Who managed the Destiny's Child group?

Her father, Mathew Knowles, managed the group.

9. When did Beyoncé rise to fame?

She rose to fame in the late 1990s.

10. What role did Beyoncé have in Destiny's Child?  

She was the lead singer of Destiny's Child.

11. What was the first album Beyoncé released as a solo artist?

Dangerously in Love

12. When did Beyoncé release Dangerously in Love?  

2003

13. How many Grammy awards did Beyoncé win for her first solo album?

Five Grammy Awards

14. What was Beyoncé's role in Destiny's Child?

She was the lead singer of the group.  

15. What was the name of Beyoncé's first solo album?  

Dangerously in Love

<br>You can see the pictures of interaction with claude 2 in the link:
https://drive.google.com/drive/folders/111jspKOhyvRvClOTLV4Aj8tli--pKaby?usp=sharing

### 3. Results analysis and evaluation

**3.1 Manually check if the output answer for each data sample is correct or not**

In [7]:
# Getting the ground truth of the selected 15 question-answer pair
ground_truth = [item['text'][0] for item in selected_data['answers']]
ground_truth

['in the late 1990s',
 'singing and dancing',
 '2003',
 'Houston, Texas',
 'late 1990s',
 "Destiny's Child",
 'Dangerously in Love',
 'Mathew Knowles',
 'late 1990s',
 'lead singer',
 'Dangerously in Love',
 '2003',
 'five',
 'lead singer',
 'Dangerously in Love']

In [8]:
# Define the elements of result table
result_table_elements = {
    'Answer for' : ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15'],
    'Ground truth' : ['in the late 1990s',
                      'singing and dancing',
                      '2003',
                      'Houston, Texas',
                      'late 1990s',
                      "Destiny's Child",
                      'Dangerously in Love',
                      'Mathew Knowles',
                      'late 1990s',
                      'lead singer',
                      'Dangerously in Love',
                      '2003',
                      'five',
                      'lead singer',
                      'Dangerously in Love'],
    "RoBERTa's answers" : ['late 1990s',
                           'singing and dancing',
                           '2003',
                           'Texas',
                           '1990s',
                           "Destiny's Child",
                           'Dangerously in Love',
                           'Mathew Knowles',
                           'late 1990s',
                           'lead singer',
                           'Dangerously in Love',
                           '2003',
                           'five',
                           'lead singer',
                           'Dangerously in Love'],
    "ChatGPT's answers" : ['Beyoncé started becoming popular in the late 1990s.',
                           'Beyoncé competed in various singing and dancing competitions as a child.',
                           "Beyoncé left Destiny's Child during their hiatus, and her debut solo album, Dangerously in Love, was released in 2003.",
                           'Beyoncé grew up in Houston, Texas.',
                           'Beyoncé became famous in the late 1990s.',
                           "Beyoncé was the lead singer of the R&B girl-group Destiny's Child.",
                           'Dangerously in Love (2003) made Beyoncé a worldwide known artist.',
                           "Destiny's Child was managed by her father, Mathew Knowles.",
                           'Beyoncé rose to fame in the late 1990s.',
                           "Beyoncé was the lead singer of Destiny's Child.",
                           "Beyoncé's first solo album was Dangerously in Love (2003).",
                           'Beyoncé released Dangerously in Love in 2003.',
                           'Beyoncé won five Grammy Awards for her first solo album, Dangerously in Love.',
                           "Beyonce was the lead singer of Destiny's Child.",
                           "Beyonce's first solo album was titled Dangerously in Love."],
    "Claude 2's answers" : ["She rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child.",
                            'The context states she performed in various singing and dancing competitions as a child.',
                            "The context states that during Destiny's Child's hiatus, Beyonce released her debut solo album Dangerously in Love in 2003. So she left the group and went solo in 2003",
                            'She was born and raised in Houston, Texas.',
                            'She became famous in the late 1990s.',
                            "Destiny's Child",
                            """Her debut solo album "Dangerously in Love" in 2003 established her as a solo artist worldwide""",
                            'Her father, Mathew Knowles, managed the group.',
                            'She rose to fame in the late 1990s.',
                            "She was the lead singer of Destiny's Child.",
                            'Dangerously in Love',
                            '2003',
                            'Five Grammy Awards',
                            'She was the lead singer of the group.',
                            'Dangerously in Love']
}

# Print the result table
result_table = pd.DataFrame(result_table_elements)
result_table

,Answer for,Ground truth,RoBERTa's answers,ChatGPT's answers,Claude 2's answers
0,Q1,in the late 1990s,late 1990s,Beyoncé started becoming popular in the late 1...,She rose to fame in the late 1990s as lead sin...
1,Q2,singing and dancing,singing and dancing,Beyoncé competed in various singing and dancin...,The context states she performed in various si...
2,Q3,2003,2003,Beyoncé left Destiny's Child during their hiat...,The context states that during Destiny's Child...
3,Q4,"Houston, Texas",Texas,"Beyoncé grew up in Houston, Texas.","She was born and raised in Houston, Texas."
4,Q5,late 1990s,1990s,Beyoncé became famous in the late 1990s.,She became famous in the late 1990s.
5,Q6,Destiny's Child,Destiny's Child,Beyoncé was the lead singer of the R&B girl-gr...,Destiny's Child
6,Q7,Dangerously in Love,Dangerously in Love,Dangerously in Love (2003) made Beyoncé a worl...,"Her debut solo album ""Dangerously in Love"" in ..."
7,Q8,Mathew Knowles,Mathew Knowles,"Destiny's Child was managed by her father, Mat...","Her father, Mathew Knowles, managed the group."
8,Q9,late 1990s,late 1990s,Beyoncé rose to fame in the late 1990s.,She rose to fame in the late 1990s.
9,Q10,lead singer,lead singer,Beyoncé was the lead singer of Destiny's Child.,She was the lead singer of Destiny's Child.


From the results in the above table, we can see that all 3 models can correctly answer all questions, thus the general score to answer the questions of these 3 models are 1 or 100% (From #correct answers/15).

**3.2 Provide comparison discussions and analysis based on the output and evaluation results from each model**

As we can see in 3.1, all three models provided 100% correct answers for all questions, but they provided answers in different ways
- **RoBERTa:** The answers generated by RoBERTa are almost exact to the ground truth answers, just a little different in some words.
- **ChatGPT:** The answers generated by ChatGPT are very different from ground truth. ChatGPT doesn't provide only the answer but also clearly show where the answers mentioned in the context.
- **Claude 2:** The answers generated by Claude 2 are quite different to both RoBERTa and ChatGPT. Claude 2 provides a little explanation about the answers but not all answers are clearly explained as ChatGPT.

In my opinion, I think ChatGPT performs best because it provides very clear answers and explanation for all questions, and RoBERTa performs worst because it only provided answers without any explanation to support those answers.

For the advantages and disadvantages of the pre-trained language models and the large language models (LLMs) based on this task,

**Pre-trained language models:**
- Advantages: This model can understand the context and correctly answer very well.
- Disadvantages: This model cannot provide a clear explanation for its answers as same as LLMs, and if the context is very unique, the model might not be able to understand the context.

**Large language models:**
- Advantages: This model can also understand the context and correctly answer very well as same as the pre-trained language models but not only provide an answer, this model also can provide a clear explanation for its answer.
- Disadvantages: This model tries to understand the context more than pre-trained language models do, so the model is more complex than pre-trained language models. So, it takes longer time to generate its answers.

**3.3 Comparison between 2 data samples**

**First sample**
<br>Questions: When did Beyonce start becoming popular?
<br>Ground truth: in the late 1990s
<br>RoBERTa's answer: late 1990s
<br>ChatGPT's answer: Beyoncé started becoming popular in the late 1990s.
<br>Claude 2's answer: She rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child.

**Second sample**
<br>Questions: In which decade did Beyonce become famous?
<br>Ground truth: late 1990s
<br>RoBERTa's answer: 1990s
<br>ChatGPT's answer: Beyoncé became famous in the late 1990s.
<br>Claude 2's answer: She became famous in the late 1990s.

<br>From the above 2 samples, we can see that all three models can provide correct answers compared to the ground truths. RoBERTa model only provides the exact answer for the question while other 2 models provide answers along with explanation of their answers from the context. ChatGPT gives the best explanation as we can see the exact name of the person from the context. Claude 2 gives a good explanation too but it uses "She" instead of the name of the person which is not clear as same ChatGPT.